Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = @__DIR__
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);

first(df, 5)

,actor,recipient,condition,block,trial,prosoc_left,chose_prosoc,pulled_left
,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,1,NA,0,1,2,0,1,0
2,1,NA,0,1,4,0,0,1
3,1,NA,0,1,6,1,0,0
4,1,NA,0,1,8,0,1,0
5,1,NA,0,1,10,1,1,1


Define the Stan language model

In [4]:
m_10_04 = "
data{
    int N;
    int N_actors;
    int pulled_left[N];
    int prosoc_left[N];
    int condition[N];
    int actor[N];
}
parameters{
    vector[N_actors] a;
    real bp;
    real bpC;
}
model{
    vector[N] p;
    bpC ~ normal( 0 , 10 );
    bp ~ normal( 0 , 10 );
    a ~ normal( 0 , 10 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + (bp + bpC * condition[i]) * prosoc_left[i];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m_10_04",
model=m_10_04, output_format=:mcmcchains);

Input data for cmdstan

In [6]:
m_10_04_data = Dict("N" => size(df, 1), "N_actors" => length(unique(df[:actor])),
"actor" => df[:actor], "pulled_left" => df[:pulled_left],
"prosoc_left" => df[:prosoc_left], "condition" => df[:condition]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, m_10_04_data, ProjDir, diagnostics=false,
  summary=true, CmdStanDir=CMDSTAN_HOME);
# Result rethinking
rethinking = "
      mean   sd  5.5% 94.5% n_eff Rhat
bp    0.84 0.26  0.43  1.26  2271    1
bpC  -0.13 0.29 -0.59  0.34  2949    1

a[1] -0.74 0.27 -1.16 -0.31  3310    1
a[2] 10.88 5.20  4.57 20.73  1634    1
a[3] -1.05 0.28 -1.52 -0.59  4206    1
a[4] -1.05 0.28 -1.50 -0.60  4133    1
a[5] -0.75 0.27 -1.18 -0.32  4049    1
a[6]  0.22 0.27 -0.22  0.65  3877    1
a[7]  1.81 0.39  1.22  2.48  3807    1
";


Inference for Stan model: m_10_04_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (1.3, 1.5, 1.4, 1.5) seconds, 5.8 seconds total
Sampling took (1.4, 1.3, 1.5, 1.4) seconds, 5.6 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%  N_Eff  N_Eff/s    R_hat
lp__             -261  4.9e-02     2.1   -265   -261   -259   1883      339  1.0e+00
accept_stat__    0.88  3.2e-03    0.18   0.47   0.95    1.0   3033      546  1.0e+00
stepsize__       0.32  1.5e-02   0.021   0.29   0.34   0.34    2.0     0.36  4.5e+13
treedepth__       3.5  2.7e-02    0.57    3.0    4.0    4.0    435       78  1.0e+00
n_leapfrog__       13  9.5e-02     3.8    7.0     15     15   1623      292  1.0e+00
divergent__     0.048  3.7e-03    0.21   0.00   0.00   0.00   3366      606  1.0e+00
energy__          266  7.5e-02     3.0    261    265    271   1634      294  1.0e+00
a[1]            -0.74  4.7e-03    0.26   -1.2  -0.

Update sections

In [8]:
chn2 = set_section(chn, Dict(
  :parameters => ["bp", "bpC"],
  :pooled => ["a.$i" for i in 1:7],
  :internals => ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__",
    "divergent__", "energy__"]
  )
)

Object of type Chains, with data of type 1000×16×4 Array{Float64,3}

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7
internals         = lp__, accept_stat__, stepsize__, treedepth__, n_leapfrog__, divergent__, energy__
parameters        = bp, bpC

parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8452 0.2561   0.0040 0.0045 1000
bpC -0.1444 0.2895   0.0046 0.0046 1000



Describe parameter draws

In [9]:
describe(chn2)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = bp, bpC

Empirical Posterior Estimates
─────────────────────────────────────────
parameters
      Mean    SD   Naive SE  MCSE   ESS
 bp  0.8452 0.2561   0.0040 0.0045 1000
bpC -0.1444 0.2895   0.0046 0.0046 1000

Quantiles
─────────────────────────────────────────
parameters
      2.5%   25.0%   50.0%   75.0%  97.5%
 bp -0.0359  0.6686  0.8433 1.0141 1.6580
bpC -1.3186 -0.3399 -0.1369 0.0498 0.7862



Describe pooled parameter draws

In [10]:
describe(chn2, section=:pooled)

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
pooled            = a.1, a.2, a.3, a.4, a.5, a.6, a.7

Empirical Posterior Estimates
───────────────────────────────────────────
pooled
      Mean    SD   Naive SE  MCSE   ESS
a.1 -0.7371 0.2647   0.0042 0.0051 1000
a.2 11.0734 5.2541   0.0831 0.1331 1000
a.3 -1.0526 0.2745   0.0043 0.0042 1000
a.4 -1.0561 0.2770   0.0044 0.0046 1000
a.5 -0.7398 0.2680   0.0042 0.0044 1000
a.6  0.2118 0.2659   0.0042 0.0044 1000
a.7  1.8184 0.4053   0.0064 0.0090 1000

Quantiles
───────────────────────────────────────────
pooled
      2.5%   25.0%   50.0%   75.0%   97.5% 
a.1 -1.7638 -0.9126 -0.7364 -0.5593  0.1499
a.2  2.1212  6.9489 10.0482 14.1618 33.5896
a.3 -2.1951 -1.2377 -1.0477 -0.8596  0.0566
a.4 -2.1332 -1.2421 -1.0501 -0.8713  0.0891
a.5 -1.7463 -0.9170 -0.7378 -0.5575  0.0682
a.6 -0.8305  0.0382  0.2101  0.3864  1.3276
a.7  0.5944  1.5361  1.8093  2.0827  3.4220



Make it a DataFrame

In [11]:
df = DataFrame(chn2, [:parameters, :pooled])

,bp,bpC,a.1,a.2,a.3,a.4,a.5,a.6,a.7
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.704099,-0.0554734,-0.441181,6.16103,-1.28328,-0.892946,-0.764471,0.236985,2.04922
2,1.04039,-0.165562,-0.845937,7.15808,-1.13019,-0.949063,-0.725547,-0.282322,1.56958
3,0.919267,0.0027261,-0.743302,3.58094,-1.33831,-0.892623,-0.672495,-0.266803,1.60074
4,0.804733,-0.431937,-0.701293,12.7126,-1.00726,-1.12809,-1.0493,0.429961,2.1332
5,0.930731,-0.11623,-0.718954,13.1911,-0.974105,-0.744035,-0.416403,0.0194877,1.9442
6,0.707119,-0.433585,-1.02635,12.1008,-0.944748,-0.726685,-0.497794,0.0279475,1.925
7,0.727111,0.106501,-0.563682,18.8631,-0.756496,-0.729768,-0.406038,0.498179,1.25198
8,0.398048,0.111756,-0.950014,8.92059,-0.607216,-1.17477,-0.89552,-0.141044,2.03171
9,0.4555,0.0246933,-0.889058,7.94327,-0.538212,-1.20306,-0.823306,-0.160876,1.94403


End of `10/m10.04s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*